In [1]:
import pandas as pd
import numpy as np
import re
from collections import Counter
import string
from sklearn.feature_extraction import stop_words

In [2]:
data = pd.read_csv('GSAF5.csv', encoding = 'latin')
data.head(5)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993,NaN,NaN
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992,NaN,NaN
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991,NaN,NaN
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,...,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990,NaN,NaN
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,...,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989,NaN,NaN


In [3]:
def supprespace(x):
    if type(x) is str:
        return x.strip()
    return x
data = data.applymap(supprespace)

# suppression des éventuelles lignes redondantes
data.drop_duplicates()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993,NaN,NaN
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992,NaN,NaN
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991,NaN,NaN
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,...,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990,NaN,NaN
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,...,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989,NaN,NaN
5,2016.09.15.R,15-Sep-16,2016,Boat,AUSTRALIA,Western Australia,Bunbury,Fishing,Occupant: Ben Stratton,NaN,...,NaN,"West Australian, 9/15/2016",2016.09.15.R-boat.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.15.R,2016.09.15.R,5988,NaN,NaN
6,2016.09.11,11-Sep-16,2016,Unprovoked,USA,Florida,"Ponte Vedra, St. Johns County",Wading,male,M,...,3' to 4' shark,"News4Jax, 9/11/2016",2016.09.11-PonteVedra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.11,2016.09.11,5987,NaN,NaN
7,2016.09.07,07-Sep-16,2016,Unprovoked,USA,Hawaii,"Makaha, Oahu",Swimming,female,F,...,"Tiger shark, 10?","Hawaii News Now, 9/7/2016",2016.09.07-Oahu.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.07,2016.09.07,5986,NaN,NaN
8,2016.09.06,06-Sep-16,2016,Unprovoked,NEW CALEDONIA,North Province,Koumac,Kite surfing,David Jewell,M,...,NaN,"TVANouvelles, 9/6/2016",2016.09.06-Jewell.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.06,2016.09.06,5985,NaN,NaN
9,2016.09.05.b,05-Sep-16,2016,Unprovoked,USA,South Carolina,"Kingston Plantation, Myrtle Beach, Horry County",Boogie boarding,Rylie Williams,F,...,NaN,"C. Creswell, GSAF",2016.09.05.b-Williams.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.05.b,2016.09.05.b,5984,NaN,NaN


In [4]:
Counter(data['Date']).most_common()

[('1957', 11),
 ('1942', 9),
 ('1956', 8),
 ('1958', 7),
 ('1950', 7),
 ('1941', 7),
 ('1949', 6),
 ('No date', 6),
 ('05-Oct-03', 5),
 ('12-Apr-01', 5),
 ('28-Jul-95', 5),
 ('1970s', 5),
 ('Oct-60', 5),
 ('1959', 5),
 ('Aug-56', 5),
 ('1955', 5),
 ('1954', 5),
 ('1940', 5),
 ('No date, Before 1963', 5),
 ('20-Sep-15', 4),
 ('28-Dec-14', 4),
 ('14-Jun-12', 4),
 ('09-Jan-10', 4),
 ('27-Dec-08', 4),
 ('1995', 4),
 ('09-Jul-94', 4),
 ('23-Jan-70', 4),
 ('1961', 4),
 ('1960', 4),
 ('1960s', 4),
 ('27-Jul-52', 4),
 ('1952', 4),
 ('1945', 4),
 ('1938', 4),
 ('Reported 10-Oct-1906', 4),
 ('1898', 4),
 ('1890', 4),
 ('1876', 4),
 ('Before 1958', 4),
 ('Before 1906', 4),
 ('18-Sep-16', 3),
 ('07-Jul-16', 3),
 ('17-Oct-15', 3),
 ('26-Jun-15', 3),
 ('24-Jun-15', 3),
 ('27-Aug-14', 3),
 ('09-Jun-14', 3),
 ('01-Jun-14', 3),
 ('14-May-14', 3),
 ('18-Mar-14', 3),
 ('01-Sep-2013', 3),
 ('28-Jul-13', 3),
 ('25-Jun-13', 3),
 ('02-Sep-12', 3),
 ('07-Jul-12', 3),
 ('26-Jun-12', 3),
 ('17-Aug-11', 3),
 ('2

In [5]:
print(data['Case Number'].count())

5992


In [6]:
#vérifie qu'il n'y ait pas de colonnes vides
null_cols = data.isnull().sum()
pourcent_col = null_cols / data['Case Number'].count()*100
print("Poids des valeurs nulles dans chacun des champs")
print(pourcent_col)
 #print(str(null_cols[null_cols > 0]))

Poids des valeurs nulles dans chacun des champs
Case Number                0.000000
Date                       0.000000
Year                       0.000000
Type                       0.000000
Country                    0.717623
Area                       6.708945
Location                   8.277704
Activity                   8.795060
Name                       3.337784
Sex                        9.462617
Age                       44.742991
Injury                     0.450601
Fatal (Y/N)                0.317089
Time                      53.621495
Species                   48.965287
Investigator or Source     0.250334
pdf                        0.000000
href formula               0.016689
href                       0.050067
Case Number.1              0.000000
Case Number.2              0.000000
original order             0.000000
Unnamed: 22               99.983311
Unnamed: 23               99.966622
dtype: float64


In [7]:
#checke le contenu des colonnes presque vides
data[data['Unnamed: 22'].fillna(0) != 0]

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
1247,2006.06.18,18-Jun-06,2006,Unprovoked,BRAZIL,Pernambuco,"Punta Del Chifre Beach, Olinda",Body boarding,Humberto Pessoa Batista,M,...,NaN,globalsurfnews.com,2006.06.18-Batista.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2006.06.18,2006.06.18,4746,stopped here,NaN


In [8]:
data[data['Unnamed: 23'].fillna(0) != 0]

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
4182,1952.07.13,13-Jul-52,1952,Provoked,USA,California,"San Diego, San Diego County",Fishing,"Gerald Howard, on board sportsfishing boat Ter...",M,...,NaN,"L.A. Times, 7/14/1952",1952.07.13-Howard.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1952.07.13,1952.07.13,1811,NaN,Teramo
5585,1879.03.10,10-Mar-1879,1879,Invalid,AUSTRALIA,New South Wales,Near Sydney,The steamship Bonnie Dundee lost in collision,Cabin boy of the Bonnie Dundee,M,...,NaN,"Star, 3/22/1879",1879.03.10-Bonnie-Dundee.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1879.03.10,1879.03.10,408,NaN,change filename


In [16]:
#vérifier les différences entre les colonnes 'href formula' et 'href'
data[data['href formula'].eq(data['href']) == False]

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
20,2016.07.29,29-Jul-16,2016,Unprovoked,SPAIN,Alicante Province,Arenales del Sol,Swimming,male,M,...,Blue shark,"Informacion.es, 7/29/2016",2016.07.29-Spain.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.07.29,2016.07.29,5973,NaN,NaN
27,2016.07.23.a,23-Jul-16,2016,Unprovoked,BAHAMAS,Abaco Islands,Green Turtle Cay,Spearfishing,Steve Cutbirth,M,...,"Bull shark, 6'","KWTX, 7/23/2016",2016.07.23.a-Cutbirth.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.07.23.a,2016.07.23.a,5966,NaN,NaN
61,2016.05.21.a,21-May-16,2016,Unprovoked,USA,Florida,"Hugenot Beach , Jacksonville, Duval County",Swimming,female,F,...,NaN,"Action News Jax, 5/23/2016",2016.05.21.a-Girl.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.05.21.a,2016.05.21.a,5932,NaN,NaN
107,2015.12.21.a,21-Dec-15,2015,Unprovoked,BRAZIL,Pernambuco,Fernano de Noronha,Scuba diving,Márcio de Castro Palma,M,...,"Tiger shark, 1.5 m","Fox News, 12/22/2015",2015.12.21.a-Brazil.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2015.12.21.a,2015.12.21.a,5886,NaN,NaN
114,2015.11.15.a,15-Nov-15,2015,Unprovoked,USA,Florida,"Ocean Reef Park, Singer Island, Palm Beach County",Surfing,Allen Engelman,M,...,"Spinner shark, 7'","5WPTV, 11/15/2015",2015.11.15.a-Engelman.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2015.11.15.a,2015.11.15.a,5878,NaN,NaN
134,2015.10.05.a,05-Oct-15,2015,Unprovoked,USA,Texas,Galveston,Wading,Gregory Slaughter,M,...,4' to 5' shark,"Houston Chronicle, 10/5/2015",2015.10.05.a-Slaughter.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2015.10.05.a,2015.10.05.a,5858,NaN,NaN
180,2015.06.27.a,27-Jun-15,2015,Unprovoked,SOUTH AFRICA,Western Cape Province,Buffels Bay near Knysna,Body Boarding,Caleb Swanepoel,M,...,White shark,"NSRI, 6/27/205",2015.06.27.a-Swanepoel.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2015.06.27.a,2015.06.27.a,5812,NaN,NaN
232,2015.01.24,24-Jan-15,2015,Unprovoked,AUSTRALIA,New South Wales,Flat Rock,Surfing,Hamish Murray,M,...,NaN,"Northern Star, 1/26/2015",2015.01.24-Murray.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2015.01.24,2015.01.24,5760,NaN,NaN
262,2014.10.19,19-Oct-14,2014,Boat,USA,California,"Leadbetter Beach, Santa Barbara County",Canoeing,Tara Burnley,F,...,NaN,"R. Collier, GSAF",2014.10.19-Tara.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2014.10.19,2014.10.19,5731,NaN,NaN
263,2014.10.18,18-Oct-14,2014,Unprovoked,USA,Hawaii,"Maalaea, South Maui",Surfing,Kaleo Roberson,M,...,12' to 14' shark,"Star Advertiser, 10/18/2014",2014.10.18-Roberson.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2014.10.18,2014.10.18,5730,NaN,NaN


In [24]:
# remplace les NaN par les valeurs du champ comparable
data['href'] = data['href'].fillna(data['href formula'])
data['href formula'] = data['href formula'].fillna(data['href'])

In [55]:
# remplace les valeurs différentes de http:... par valeur correctes du champ équivalent
# dans un sens...
dataf = data['href'].apply(lambda x: not x.startswith('http'))
values = data[dataf]
data.loc[values.index, 'href'] = data.loc[values.index, 'href formula']
#... puis dans l'autre
dataf1 = data['href formula'].apply(lambda x: not x.startswith('http'))
values = data[dataf1]
data.loc[values.index, 'href formula'] = data.loc[values.index, 'href']

ErrRestantes = data[data['href formula'].eq(data['href']) == False]
print(ErrRestantes[['href','href formula']])

                                                   href  \
20    http://sharkattackfile.net/spreadsheets/pdf_di...   
27    http://sharkattackfile.net/spreadsheets/pdf_di...   
61    http://sharkattackfile.net/spreadsheets/pdf_di...   
107   http://sharkattackfile.net/spreadsheets/pdf_di...   
114   http://sharkattackfile.net/spreadsheets/pdf_di...   
134   http://sharkattackfile.net/spreadsheets/pdf_di...   
180   http://sharkattackfile.net/spreadsheets/pdf_di...   
232   http://sharkattackfile.net/spreadsheets/pdf_di...   
262   http://sharkattackfile.net/spreadsheets/pdf_di...   
263   http://sharkattackfile.net/spreadsheets/pdf_di...   
264   http://sharkattackfile.net/spreadsheets/pdf_di...   
271   http://sharkattackfile.net/spreadsheets/pdf_di...   
272   http://sharkattackfile.net/spreadsheets/pdf_di...   
293   http://sharkattackfile.net/spreadsheets/pdf_di...   
305   http://sharkattackfile.net/spreadsheets/pdf_di...   
323   http://sharkattackfile.net/spreadsheets/pdf_di... 

In [56]:
dataf = data['href'].apply(lambda x: x.startswith('http://sharkattackfile.net/spreadsheets/pdf_directory/http://sharkattackfile.net/spreadsheets/pdf_directory/'))
values = data[dataf]
data.loc[values.index, 'href'] = data.loc[values.index, 'href formula']
data[data['href formula'].eq(data['href']) == False]
#dataf2 = data['href formula'].eq(data['href']) == False
#dataf3 = dataf2.iloc[:,16:19]
#dataf3.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
27,2016.07.23.a,23-Jul-16,2016,Unprovoked,BAHAMAS,Abaco Islands,Green Turtle Cay,Spearfishing,Steve Cutbirth,M,...,"Bull shark, 6'","KWTX, 7/23/2016",2016.07.23.a-Cutbirth.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.07.23.a,2016.07.23.a,5966,NaN,NaN
134,2015.10.05.a,05-Oct-15,2015,Unprovoked,USA,Texas,Galveston,Wading,Gregory Slaughter,M,...,4' to 5' shark,"Houston Chronicle, 10/5/2015",2015.10.05.a-Slaughter.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2015.10.05.a,2015.10.05.a,5858,NaN,NaN
180,2015.06.27.a,27-Jun-15,2015,Unprovoked,SOUTH AFRICA,Western Cape Province,Buffels Bay near Knysna,Body Boarding,Caleb Swanepoel,M,...,White shark,"NSRI, 6/27/205",2015.06.27.a-Swanepoel.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2015.06.27.a,2015.06.27.a,5812,NaN,NaN
263,2014.10.18,18-Oct-14,2014,Unprovoked,USA,Hawaii,"Maalaea, South Maui",Surfing,Kaleo Roberson,M,...,12' to 14' shark,"Star Advertiser, 10/18/2014",2014.10.18-Roberson.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2014.10.18,2014.10.18,5730,NaN,NaN
264,2014.10.17,17-Oct-14,2014,Unprovoked,AUSTRALIA,New South Wales,Avoca Beach,Surfing,Kirra-Belle Olsson,F,...,1 m shark,"Sydney Morning Herald, 10/17/2014","2014.10.17-Olsson, pdf",http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2014.10.17,2014.10.17,5729,NaN,NaN
271,2014.10.03.b,03-Oct-14,2014,Boat,USA,California,Santa Barbara County,Kayaking,Ryan Howell,M,...,"White shark, 18' to 20'","R. Collier, GSAF",2014.10.03.b-Kayaks2.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2014.10.03.b,2014.10.03.b,5721,NaN,NaN
272,2014.10.03.a,03-Oct-14,2014,Boat,USA,California,Santa Barbara County,Kayaking,Raul Armenta,M,...,White shark,"R. Collier, GSAF",2014.10.03.a-Kayaks1.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2014.10.03.a,2014.10.03.a,5720,NaN,NaN
293,2014.08.12,12-Aug-14,2014,Unprovoked,USA,Florida,"3 to 4 miles west of Indian Pass, Gulf County",Standing,Kyle Hayes,M,...,"Bull shark, 8'","K. Hayes / Brewton Standard, 8/19/2014",2014.08.15-Hayes.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2014.08.12,2014.08.12,5699,NaN,NaN
323,2014.06.09.a,09-Jun-14,2014,Unprovoked,AUSTRALIA,South Australia,"Parsons Beach, Fleurieu Peninsula",Body boarding,Scott Berry,M,...,"Bronze whaler shark, 1.5m","The Australian, 6/9/2014",2014.06.09.a-Berry.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2014.06.09.a,2014.06.09.a,5669,NaN,NaN
361,2014.02.07.b,07-Feb-14,2014,Provoked,TRINIDAD & TOBAGO,Trinidad,NaN,Fishing,Simon Torres,M,...,NaN,"Trinidad Guardian, 2/11/2014",2014.02.07.b-Torres.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2014.02.07.b,2014.02.07.b,5632,NaN,NaN


In [65]:
dataf = data['href'].apply(lambda x: not x.endswith('.pdf'))
data[dataf == True]
#data.loc[values.index, 'href'] = data.loc[values.index, 'href formula']
#data[data['href formula'].eq(data['href']) == False]

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
65,2016.05.02,02-May-16,2016,Provoked,NEW ZEALAND,North Island,Cormandel,Fishing,male,M,...,"Mako shark, 1.5 m [5']",Radio New Zealand 5/3/2016,2016.05.02-NZ-pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.05.02,2016.05.02,5928,NaN,NaN
126,2015.10.17.b,17-Oct-15,2015,Invalid,USA,Hawaii,"Waikiki,",Surfing,male,M,...,No shark involvement,"KHON2, 10/17/2015",2015.10.17.b.-Hawaii. pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2015.10.17.b,2015.10.17.b,5866,NaN,NaN
157,2015.08.19,19-Aug-15,2015,Unprovoked,USA,Florida,"Jacksonville Beach, Duval County",Walking,Kaley Szarmack,F,...,3' shark,"ABC News, 8/21/2015",2015.08.19-Szsarmack,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2015.08.19,2015.08.19,5835,NaN,NaN
177,2015.06.30.b,30-Jun-15,2015,Unprovoked,AUSTRALIA,New South Wales,"Flat Rock, Yamba",Surfing,Steve,M,...,NaN,"The Telegraph, 7/8/2015",2015.06.30.b-Steve.pf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2015.06.30.b,2015.06.30.b,5815,NaN,NaN
193,2015.06.13,13-Jun-15,2015,Unprovoked,USA,California,Off San Diego,NaN,Elke Specker,F,...,Mako shark,"Courthouse News Service, 11/18/2015",Court Case pending,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2015.06.13,2015.06.13,5799,NaN,NaN
263,2014.10.18,18-Oct-14,2014,Unprovoked,USA,Hawaii,"Maalaea, South Maui",Surfing,Kaleo Roberson,M,...,12' to 14' shark,"Star Advertiser, 10/18/2014",2014.10.18-Roberson.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2014.10.18,2014.10.18,5730,NaN,NaN
295,2014.08.09.R,09-Aug-14,2014,Unprovoked,BAHAMAS,NaN,NaN,Spearfishing,Andrew Hindley,M,...,"Bull shark, 9' to 10'",A. Hindley / M. Michaelson,"2014.08.09.R-Hindley, pdf",http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2014.08.09.R,2014.08.09.R,5697,NaN,NaN
334,2014.05.14,14-May-14,2014,Unprovoked,AUSTRALIA,South Australia,"Elliston, Eyre Peninsula",Surfing,Andrew McLeod,M,...,"white shark, 15'","N. Davies, The Advertiser, 5/114/2014","2014.05.14-McLeod, pdf",http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2014.05.14,2014.05.14,5659,NaN,NaN
343,2014.04.12.R,Reported 12-Apr-2014,2014,Boat,SOUTH AFRICA,NaN,NaN,Shark watching,Inflatable boat,NaN,...,White shark,"You Tube, posted 4/12/2014",.2014.04.12.R-inflatable,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2014.04.12.R,2014.04.12.R,5650,NaN,NaN
352,2014.03.18.c,18-Mar-14,2014,Unprovoked,NEW CALEDONIA,Baie de Sainte-Marie,NaN,Kite Surfing,Laurent Borgna,M,...,NaN,"A. Brenneka, Shark Attack Surviors",2014.03.18.c-Borgna,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2014.03.18.c,2014.03.18.c,5641,NaN,NaN


In [12]:
## HELP
# remplace les NaN par les valeurs du champ comparable
#map(lambda x:x.startswith ('http'), data['href'])
#data[data['href formula'].eq(data['href']) == False]

data[data['href'].apply(lambda x:x.startswith('http')) == False]


,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
365,2014.01.04,04-Jan-14,2014,Sea Disaster,JAPAN,Okinawa Prefecture,Off Miyako Island,Sea disaster,Rianto,M,...,NaN,"Focus Taiwan, 1/7/2014",2014.01.04-Rianto.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,pdf_directory2014.01.04-Riano.pdf,2014.01.04,2014.01.04,5628,NaN,NaN
5906,ND.0100,2003?,0,Unprovoked,BAHAMAS,Andros Islands,Great Guana Cay,Spearfishing,C.D. Dollar,M,...,1.8 m [6'] shark,"R.D. Weeks, GSAF",ND-0100-CDDollar.pdf,Q93http://sharkattackfile.net/spreadsheets/pdf...,Q93http://sharkattackfile.net/spreadsheets/pdf...,ND.0100,ND.0100,87,NaN,NaN


In [132]:
data[data['href'].isnull()]

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23


In [75]:
corr1 = data[data['Case Number'].eq(data['Case Number.1']) == False]
corr2 = data[data['Case Number'].eq(data['Case Number.2']) == False]
listeerr = [corr1, corr2]
corrCaseNum = pd.concat(listeerr)
corrCaseNum[['Case Number', 'Case Number.1','Case Number.2', 'Date', 'Year']]


,Case Number,Case Number.1,Case Number.2,Date,Year
4,2016.09.15,2016.09.16,2016.09.15,16-Sep-16,2016
33,2016.07.14.4,2016.07.14.R,2016.07.14.4,Reported 14-Jul-2016,2016
97,2016.01.24.b,2015.01.24.b,2016.01.24.b,24-Jan-16,2016
116,2015.12.23,2015.11.07,2015.12.23,07-Nov-15,2015
121,2015.10.28.a,2015.10.28,2015.10.28.a,28-Oct-15,2015
169,2015.07-10,2015.07.10,2015.07.10,10-Jul-15,2015
3296,1967.07.05,1967/07.05,1967.07.05,05-Jul-67,1967
3569,"1962,08.30.b",1962.08.30.b,"1962,08.30.b",30-Aug-62,1962
3654,1961.09.02.R,"1961.09,06.R",1961.09.02.R,Reported 06-Sep-1961,1961
4177,1952.08.05,1952.08.04,1952.08.05,05-Aug-52,1952


In [ ]:
for i in corrCaseNum:
    if 

In [53]:
#supprime les colonnes inutiles
data.drop(['Unnamed: 22', 'Unnamed: 23', 'href formula'], axis = 1, inplace = True)

In [125]:
#null_cols = data.isnull().sum()
#null_cols[null_cols > 0]
def num_missing(x):
    return sum(x.isnull())
print("Valeurs manquantes par colonne :")
print(data.apply(num_missing, axis=0))

Valeurs manquantes par colonne :
Case Number                  0
Date                         0
Year                         0
Type                         0
Country                     43
Area                       402
Location                   496
Activity                   527
Name                       200
Sex                        567
Age                       2681
Injury                      27
Fatal (Y/N)                 19
Time                      3213
Species                   2934
Investigator or Source      15
pdf                          0
href formula                 1
href                         3
Case Number.1                0
Case Number.2                0
original order               0
Unnamed: 22               5991
Unnamed: 23               5990
dtype: int64


In [181]:
data.iloc[57,:]

Case Number                                                    2016.05.29.b
Date                                                              29-May-16
Year                                                                   2016
Type                                                             Unprovoked
Country                                                                 USA
Area                                                             California
Location                             Corona Del Mar, Newport, Orange County
Activity                                                           Swimming
Name                                                    Maria Korcsmaros \n
Sex                                                                       F
Age                                                                      52
Injury                                         Injuries to arm and shoulder
Fatal (Y/N)                                                               N
Time        

In [169]:
data.dtypes

Case Number               object
Date                      object
Year                       int64
Type                      object
Country                   object
Area                      object
Location                  object
Activity                  object
Name                      object
Sex                       object
Age                       object
Injury                    object
Fatal (Y/N)               object
Time                      object
Species                   object
Investigator or Source    object
pdf                       object
href formula              object
href                      object
Case Number.1             object
Case Number.2             object
original order             int64
Unnamed: 22               object
Unnamed: 23               object
datecorr                  object
test                      object
dtype: object

In [171]:
data['Age'] = data['Age'].fillnull(0).astype('int')

AttributeError: 'Series' object has no attribute 'fillnull'

In [32]:

### HELP
#data['Date'].str.contains('[a-z]').counter()
#data['Date'] = data['Date'].astype (str)
#re.findall('[a-z]*', data['Date'])
#pd.to_datetime(data['Date'])
#data['Date'].dtype
'''
def convert_date(date):
    date6pos = data['Date'].str.extract(r'([0-9]{2}-[A-Za-z]{3}-[0-9]{2})')
    if date6pos.match(date):
        l = date.split('-')
        return l[0] + '-' + l[1] + '-' + l[2]

data['formatted_date'] = data.Date.apply(convert_data)
'''
for i in data['Date'] :
    j = data['Date'][i].str.extract(r'([0-9]{2}-[A-Za-z]{3}-[0-9]{4})')
    data['datecorr'].append(j)
print(data['datecorr'])    
#data['datecorr'] = data['Date'].str.extract(r'([0-9]{2}-[A-Za-z]{3}-[0-9]{2})')
#data['datecorr'] = data['datecorr'].astype(str)


#def test(date):
#    l = date.split('-')
#    return l[0] + '-' + l[1] + '-19'
#data['datecorr'] = data['datecorr'].dropna().apply(test)

'''
data['datecorr'] = convert_date(data['Date'])

Counter(data['datecorr']).most_common()
'''

KeyError: '18-Sep-16'

In [34]:
re.findall(r'([0-9]{2}-[A-Za-z]{3}-[0-9]{4})',str(data['Date']))

TypeError: expected string or bytes-like object

In [208]:
data['datecorr2'] = data['Date'].str.extract(r'([0-9]{2}-[A-Za-z]{3}-[0-9]{4})')
Counter(data['datecorr2']).most_common()

#data ['datecorr3'] = '01-Jan-' + data['Date'].str.extract(r'([0-9]{4})')
#Counter(data['datecorr3']).most_common()

[(nan, 5201),
 ('10-Oct-1906', 4),
 ('01-Sep-2013', 3),
 ('08-Aug-1899', 3),
 ('26-Jul-1885', 3),
 ('21-Sep-2013', 2),
 ('07-Sep-2013', 2),
 ('25-Aug-2013', 2),
 ('29-Jul-2013', 2),
 ('29-Oct-2011', 2),
 ('19-Jul-2007', 2),
 ('13-Jun-2002', 2),
 ('05-Nov-1997', 2),
 ('19-Aug-1993', 2),
 ('26-Jun-1972', 2),
 ('24-Jan-1920', 2),
 ('06-Jul-1915', 2),
 ('27-Aug-1913', 2),
 ('16-Oct-1907', 2),
 ('12-Aug-1907', 2),
 ('27-Sep-1906', 2),
 ('12-Oct-1899', 2),
 ('19-Sep-1898', 2),
 ('15-Mar-1897', 2),
 ('15-Jun-1894', 2),
 ('28-Apr-1894', 2),
 ('22-Jun-1893', 2),
 ('08-Feb-1887', 2),
 ('26-Nov-1885', 2),
 ('12-Nov-1882', 2),
 ('02-May-1880', 2),
 ('02-Sep-1878', 2),
 ('15-Dec-1877', 2),
 ('15-Jul-1874', 2),
 ('15-Jun-1874', 2),
 ('08-Jun-1849', 2),
 ('19-Jul-1913', 2),
 ('14-Jul-2016', 1),
 ('08-Jul-2016', 1),
 ('03-Mar-2016', 1),
 ('19-Feb-2016', 1),
 ('10-Feb-2016', 1),
 ('11-Jan-2016', 1),
 ('25-Jun-2015', 1),
 ('23-Dec-2014', 1),
 ('03-Dec-2014', 1),
 ('17-Nov-2014', 1),
 ('12-Sep-2014', 1),

In [160]:
data.describe(include='all').transpose

<bound method DataFrame.transpose of        Case Number  Date         Year        Type Country     Area  \
count         5992  5992  5992.000000        5992    5949     5590   
unique        5976  5128          NaN           6     203      785   
top     2006.09.02  1957          NaN  Unprovoked     USA  Florida   
freq             2    11          NaN        4386    2116      990   
mean           NaN   NaN  1925.204606         NaN     NaN      NaN   
std            NaN   NaN   286.473712         NaN     NaN      NaN   
min            NaN   NaN     0.000000         NaN     NaN      NaN   
25%            NaN   NaN  1942.000000         NaN     NaN      NaN   
50%            NaN   NaN  1975.000000         NaN     NaN      NaN   
75%            NaN   NaN  2003.000000         NaN     NaN      NaN   
max            NaN   NaN  2016.000000         NaN     NaN      NaN   

                                Location Activity  Name  Sex   ...  \
count                               5496     5465  5

In [126]:
Counter(data['Injury']).most_common()

[('FATAL', 729),
 ('Survived', 97),
 ('Foot bitten', 82),
 ('No injury', 77),
 ('Leg bitten', 71),
 ('Left foot bitten', 48),
 ('No details', 43),
 ('Right foot bitten', 39),
 ('Hand bitten', 28),
 ('Thigh bitten', 27),
 ('No injury, board bitten', 27),
 (nan, 27),
 ('FATAL, body not recovered', 24),
 ('Minor injury', 22),
 ('Calf bitten', 21),
 ('Foot lacerated', 21),
 ('Right leg bitten', 20),
 ('Arm bitten', 19),
 ('Lacerations to foot', 19),
 ('Ankle bitten', 17),
 ('Right calf bitten', 16),
 ('Lacerations to right foot', 15),
 ('Lacerations to left foot', 14),
 ('No injury to occupants', 14),
 ('No injury, surfboard bitten', 13),
 ('Heel bitten', 13),
 ('Left arm bitten', 13),
 ('Left leg bitten', 13),
 ('Foot severed', 13),
 ('Right thigh bitten', 12),
 ('Leg lacerated', 12),
 ('Leg injured', 11),
 ('Left calf bitten', 11),
 ('Minor injuries', 11),
 ('Leg severed', 11),
 ('FATAL, leg severed ', 11),
 ('Lacerations to leg', 10),
 ('Lacerations to right leg', 10),
 ('Legs bitten', 

In [27]:
## HELP

#liste_traitee = re.compile('[a-z]',data['Injury'])
#print(liste_traitee)

bag_of_words = []
caract_virer = ';.!?\n'
liste = re.compile(caract_virer)
print(liste)
liste_injury = [i for i in str(data['Injury']).replace('\n',' ').replace('.',' ').replace('"',' ').replace('\W',' ').split(' ')]
list_of_words = [i.lower() for i in liste_injury if i.lower() not in stop_words.ENGLISH_STOP_WORDS]
bag_of_words = list(set(list_of_words))
print(bag_of_words)


#term_freq = []
#for word in bag_of_words:
#    term_freq.append(str(data['Injury']).count (word))
#print(term_freq)

re.compile(';.!?\n')
['', '5988', 'trolling', '12', 'ca', 'recovered', '16', 'half,', '5975', 'injury,', '20', 'half', 'injury', 'minor', 'severely', '5981', 'hands', '14', '5978', 'occupant;', '5974', 'shark', 'broken', 'collided', '5986', 'killed', 'hooked', '23', 'x', 'fatal', '11', '29', '1', 'lacerations', '5977', 'bit', 'motor', 'occupant', 'severed', '5964', 'object', '5979', '5980', '5963', 'prov', 'away', '5965', '5969', 'forearm', 'leg', 'rope', '5968', '3', '5970', '13', '24', 'dtype:', 'injury:', 'laceration', 'fatal,', '5984', '5967', 'face', 'arm', '5976', 'fin', 'inury,', '28', '19', 'bumped', '4', 'kayak', '2', 'sur', 'overboard', 'carrying', 'left', '2-inch', 'hand', 'calf', '17', 'punctures', '5971', 'incident', 'bitten', 'tail', '5987', 'foot', '5', 'lacerated', 'board', '26', '5966', '5972', 'knocked', 'people', '5989', '7', 'toes', '25', 'repeatedly', 'body', 'struck', '5985', '5992,', '8', 'big', 'length:', 'thigh', 't', 'sharks,', 'lower', 'ankle', '&', 'severed,

In [116]:
data.loc[(data['Country'] == "REUNION"), ['Country', 'Area', 'Location']]

,Country,Area,Location
15,REUNION,NaN,Boucan Canot
167,REUNION,NaN,St. Leu
197,REUNION,Le Port,Folette
214,REUNION,Saint-Gilles-les-Bains,Cap Homard
226,REUNION,dÉtang-Salé,Ravine Mula
305,REUNION,Saint-Leu,NaN
384,REUNION,dÉtang-Salé,Ravine Mula
431,REUNION,Saint-Paul,Le cimetière marin
454,REUNION,Saint-Gilles,Brisant Beach
532,REUNION,Saint Leu,NaN


In [114]:
data.loc[data['Case Number'] == None]

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
